# Using a Knowledge Graph to Implement a RAG Application

In this notebook, we'll explore how to implement a knowledge graph to augment the language generation process for RAG application. 
We'll cover the following key steps:
Loading and preprocessing text data.
Initializing a language model and extracting a knowledge graph.
Storing the knowledge graph in a database for persistence and querying.
Retrieving relevant knowledge from the graph based on user queries.
Generating responses by synthesizing the retrieved knowledge with the language model.


By the end of this notebook, you'll have a solid understanding of implementing knowledge graphs in RAG applications, enabling you to build more intelligent and context-aware language generation systems.

Prerequisites
Before we begin, make sure you have the following installed:
Python 3.7 or higher
LangChain library (`pip install langchain`)
LlamaIndex library (`pip install llama-index`)
Neo4j database (or any other compatible graph database)


## Step 1: load and preprocess text data
The first step is to load and preprocess the text data from which we'll extract the knowledge graph. In this example, we'll use a text snippet describing a technology company called prismaticAI, its employees, and their roles.

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

# Load text data
text = """Sarah is an employee at prismaticAI, a leading technology company based in Westside Valley. She has been working there for the past three years as a software engineer.

Michael is also an employee at prismaticAI, where he works as a data scientist. He joined the company two years ago after completing his graduate studies.

prismaticAI is a well-known technology company that specializes in developing cutting-edge software solutions and artificial intelligence applications. The company has a diverse workforce of talented individuals from various backgrounds.

Both Sarah and Michael are highly skilled professionals who contribute significantly to prismaticAI's success. They work closely with their respective teams to develop innovative products and services that meet the evolving needs of the company's clients."""

loader = TextLoader(text)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20)
texts = text_splitter.split_documents(documents)

We start by importing the necessary classes from LangChain: `TextLoader` and `CharacterTextSplitter`. `TextLoader` loads the text data, while `CharacterTextSplitter` splits the text into smaller chunks for more efficient processing.
Next, we define the text data as a multi-line string variable `text`.
We then use `TextLoader` to load the text data directly from the `text` variable. The `loader.load()` method returns a list of `Document` objects, each containing a chunk of the text.
To split the text into smaller, more manageable chunks, we create an instance of `CharacterTextSplitter` with a `chunk_size` of 200 characters and a `chunk_overlap` of 20 characters. The `chunk_overlap` parameter ensures that there is some overlap between adjacent chunks, which can be helpful for maintaining context during the knowledge extraction process.
Finally, we use the `split_documents` method of `CharacterTextSplitter` to split the `Document` objects into smaller chunks, which are stored in the `texts` variable as a list of `Document` objects.
Preprocessing the text data in this way allows us to prepare it for the next step, where we'll initialize a language model and use it to extract a knowledge graph from the text chunks.


## Step 2: Initialize language model and extract knowledge graph
After loading and preprocessing the text data, the next step is to initialize a language model and use it to extract a knowledge graph from the text chunks. In this example, we'll be using the OpenAI language model provided by LangChain.


In [ ]:
from langchain.llms import OpenAI
from langchain.transformers import LLMGraphTransformer
import getpass
import os

# Load environment variable for OpenAI API key
os.environ["OPENAI_API_KEY"] = getpass.getpass()

# Initialize LLM
llm = OpenAI(temperature=0)

# Extract Knowledge Graph
llm_transformer = LLMGraphTransformer(llm=llm)
graph_documents = llm_transformer.convert_to_graph_documents(texts)

First, we import the necessary classes from LangChain: `OpenAI` and `LLMGraphTransformer`. `OpenAI` is a wrapper for the OpenAI language model, which we'll use to extract the knowledge graph. `LLMGraphTransformer` is a utility class that helps convert text data into a knowledge graph representation.
Next, we load the OpenAI API key from an environment variable. This is a security best practice to avoid hardcoding sensitive credentials in your code.

We then initialize an instance of the `OpenAI` language model with a `temperature` of `0`. The temperature parameter controls the randomness of the model's output, with lower values producing more deterministic responses.
After initializing the language model, we create an instance of `LLMGraphTransformer` and pass the initialized `llm` object to it. The `LLMGraphTransformer` class converts the text chunks (`texts`) into a knowledge graph representation.
Finally, we call the `convert_to_graph_documents` method of `LLMGraphTransformer`, passing in the `texts` list. This method uses the language model to analyze the text chunks and extract relevant entities, relationships, and other structured information, which are then represented as a knowledge graph. The resulting knowledge graph is stored in the `graph_documents` variable.

We have successfully initialized a language model and used it to extract a knowledge graph from the text data. In the next step, we’ll store the knowledge graph in a database for persistence and querying.


## Step 3: Store knowledge graph in a database
After extracting the knowledge graph from the text data, it's important to store it in a persistent and queryable format. In this notebook, we'll use Neo4j to store the knowledge graph. 


In [ ]:
from langchain.graph_stores import Neo4jGraphStore

# Store Knowledge Graph in Neo4j
graph_store = Neo4jGraphStore(url="neo4j://your_neo4j_url", username="your_username", password="your_password")
graph_store.write_graph(graph_documents)

First, we import the `Neo4jGraphStore` class from LangChain. This class provides a convenient interface for interacting with a Neo4j database and storing knowledge graphs.
Next, we create an instance of `Neo4jGraphStore` by providing the necessary connection details: the Neo4j database URL, username, and password. Make sure to replace `"your_neo4j_url"`, `"your_username"`, and `"your_password"` with the appropriate values for your Neo4j instance.

Finally, we call the `write_graph` method of the `graph_store` instance, passing in the `graph_documents` list obtained from the previous step. This method serializes the knowledge graph and writes it to the Neo4j database.
Storing the knowledge graph in a Neo4j database allows us to ensure that it’s persistent and can be easily queried and retrieved when needed. The graph structure of Neo4j allows for efficient representation and traversal of the complex relationships and entities present in the knowledge graph.

In the next step, we'll set up the components for retrieving knowledge from the graph and generating responses using the retrieved context.


It's important to note that while this tutorial uses Neo4j as the graph database, LangChain supports other graph databases as well, such as Amazon Neptune and TinkerPop-compatible databases like Gremlin Server. You can swap out the `Neo4jGraphStore` with the appropriate graph store implementation for your chosen database.

## Step 4: Retrieve knowledge for RAG
Now that we have stored the knowledge graph in a database, we can set up the components for retrieving relevant knowledge from the graph based on user queries and generating responses using the retrieved context. This is the core functionality of a RAG application.

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever
from llama_index.core.response_synthesis import ResponseSynthesizer

# Retrieve Knowledge for RAG
graph_rag_retriever = KnowledgeGraphRAGRetriever(storage_context=graph_store.storage_context, verbose=True)
query_engine = RetrieverQueryEngine.from_args(graph_rag_retriever)



First, we import the necessary classes from LlamaIndex: `RetrieverQueryEngine`, `KnowledgeGraphRAGRetriever`, and `ResponseSynthesizer`.
`RetrieverQueryEngine` is a query engine that uses a retriever to fetch relevant context from a data source (in our case, the knowledge graph) and then synthesizes a response using that context.
`KnowledgeGraphRAGRetriever` is a specialized retriever that can retrieve relevant information from a knowledge graph stored in a database.
`ResponseSynthesizer` is responsible for generating a final response by combining the retrieved context with a language model.

Next, we create an instance of `KnowledgeGraphRAGRetriever` by passing in the `storage_context` from our `graph_store` instance. This `storage_context` contains the necessary information to connect to and query the Neo4j database, where we stored the knowledge graph. We also set `verbose=True` to enable detailed logging during the retrieval process.
Then, we initialize a `RetrieverQueryEngine` using the `from_args` method and passing in our `graph_rag_retriever` instance. This query engine will handle the entire process of retrieving relevant context from the knowledge graph and generating a response based on that context.

With these components set up, we are now ready to query the knowledge graph and generate responses using the retrieved context. In the next step, we'll see how to do this in practice.


## Step 5: Query the knowledge graph and generate a response
Finally, we can query the knowledge graph and generate responses using the retrieved context.

In [ ]:
def query_and_synthesize(query):
    retrieved_context = query_engine.query(query)
    response = response_synthesizer.synthesize(query, retrieved_context)
    print(f"Query: {query}")
    print(f"Answer: {response}\n")

# Initialize the ResponseSynthesizer instance
response_synthesizer = ResponseSynthesizer(llm)

# Query 1
query_and_synthesize("Where does Sarah work?")

# Query 2
query_and_synthesize("Who works for prismaticAI?")

# Query 3
query_and_synthesize("Does Michael work for the same company as Sarah?")

In this example, we define three different queries related to the employees and the company described in the text data. For each query, we use the `query_engine` to retrieve the relevant context from the knowledge graph, create an instance of `ResponseSynthesizer`, and call its `synthesize` method with the query and retrieved context.

The `ResponseSynthesizer` uses the language model and the retrieved context to generate a final response to the query, which is then printed to the console, matching the answers in the first section of this article.